## Data

In [1]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 15
sequence_length = 7

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>", "<sos_topic1>", "<sos_topic2>", "<eos>"]
                                 )
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'<unk>': 0,
 'kẻ': 8,
 '<eos>': 4,
 'cây': 6,
 'thì': 12,
 'chí': 5,
 '<pad>': 1,
 '<sos_topic1>': 2,
 '<sos_topic2>': 3,
 'có': 7,
 'nhớ': 9,
 'nên': 10,
 'quả': 11,
 'trồng': 13,
 'ăn': 14}

In [3]:
data_x = []
data_y = []

corpus[0] = '<sos_topic1> ' + corpus[0] + ' <eos>'
corpus[1] = '<sos_topic2> ' + corpus[1] + ' <eos>'

for vector in corpus:
    vector = vector.split()
    data_x.append(vector[:-1])
    data_y.append(vector[1:])  

print(data_x)
print(data_y)

[['<sos_topic1>', 'ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây'], ['<sos_topic2>', 'có', 'chí', 'thì', 'nên']]
[['ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây', '<eos>'], ['có', 'chí', 'thì', 'nên', '<eos>']]


In [4]:
# Tokenize and numericalize your samples
def vectorize(x, y, vocab, sequence_length):    
    x_ids = [vocab[token] for token in x][:sequence_length]
    x_ids = x_ids + [vocab["<pad>"]] * (sequence_length - len(x))

    y_ids = [vocab[token] for token in y][:sequence_length]
    y_ids = y_ids + [vocab["<pad>"]] * (sequence_length - len(y))
    
    return x_ids, y_ids

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, vocab, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

In [5]:
for x, y in zip(data_x_ids, data_y_ids):
    print(x)
    print(y)
    print()

[2, 14, 11, 9, 8, 13, 6]
[14, 11, 9, 8, 13, 6, 4]

[3, 7, 5, 12, 10, 1, 1]
[7, 5, 12, 10, 4, 1, 1]



In [6]:
data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
print(data_x_ids.shape)

torch.Size([2, 7])


In [7]:
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)
print(data_y_ids.shape)

torch.Size([2, 7])


## Train with full data

In [8]:
class TransformerDecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=embed_dim, 
                                          num_heads=num_heads,
                                          batch_first=True)
        self.ffn = nn.Linear(in_features=embed_dim, 
                             out_features=ff_dim)
        self.layernorm_1 = nn.LayerNorm(normalized_shape=embed_dim)
        self.layernorm_2 = nn.LayerNorm(normalized_shape=embed_dim)

    def forward(self, query, key, value, mask):
        attn_output, _ = self.attn(query, key, value, attn_mask=mask)
        out_1 = self.layernorm_1(query + attn_output)
        ffn_output = nn.GELU()(self.ffn(out_1))
        out_2 = self.layernorm_2(out_1 + ffn_output)
        return out_2

# test
transformer = TransformerDecoderBlock(6, 2, 6)
t = torch.randn(1, 10, 6)
mask = torch.triu(torch.ones(10, 10), diagonal=1).bool()  
output = transformer(t, t, t, mask)
print(output.shape)

torch.Size([1, 10, 6])


In [9]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, sequence_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim)
        self.mask = torch.triu(torch.ones(sequence_length, sequence_length), 
                               diagonal=1).bool()
        self.transformer = TransformerDecoderBlock(embed_dim, 
                                                   num_heads, 
                                                   embed_dim)
        self.linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x, x, self.mask)        
        x = self.linear(x)
        return x.permute(0,2,1)

model = TG_Model(vocab_size, 8, 1, sequence_length)
print(model)

TG_Model(
  (embedding): Embedding(15, 8)
  (transformer): TransformerDecoderBlock(
    (attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
    )
    (ffn): Linear(in_features=8, out_features=8, bias=True)
    (layernorm_1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (layernorm_2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
  )
  (linear): Linear(in_features=8, out_features=15, bias=True)
)


In [10]:
outputs = model(data_x_ids)
print(outputs.shape)

torch.Size([2, 15, 7])


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [12]:
# train
for _ in range(40):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    print(loss.item())
    loss.backward()
    optimizer.step()

2.8811280727386475
2.3223893642425537
1.895203709602356
1.5507490634918213
1.26129949092865
1.0438002347946167
0.8457573652267456
0.6351191401481628
0.4720136523246765
0.3633754253387451
0.2744462788105011
0.19876153767108917
0.14247870445251465
0.10051611810922623
0.06994500756263733
0.04941049590706825
0.0354958139359951
0.025932064279913902
0.019251320511102676
0.014383277855813503
0.010981067083775997
0.008836755529046059
0.006853047292679548
0.004778813570737839
0.0036516382824629545
0.003087684279307723
0.00268188607878983
0.0022639837116003036
0.001851539360359311
0.0015131811378523707
0.001276497496291995
0.0011303945211693645
0.00104713614564389
0.0009869779460132122
0.000912548101041466
0.0008192049572244287
0.0007278358680196106
0.0006530717364512384
0.0005961103015579283
0.000552563404198736


In [13]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=1))

tensor([[14, 11,  9,  8, 13,  6,  4],
        [ 7,  5, 12, 10,  4,  1,  1]])


In [14]:
data_y_ids

tensor([[14, 11,  9,  8, 13,  6,  4],
        [ 7,  5, 12, 10,  4,  1,  1]])

## Inference

In [15]:
promt = '<sos_topic2> có'
promt = promt.split()
promt_ids = [vocab[token] for token in promt][:sequence_length]
promt_ids = promt_ids + [vocab["<pad>"]] * (sequence_length - len(promt))

print(promt_ids)

[3, 7, 1, 1, 1, 1, 1]


In [16]:
for i in range(sequence_length - len(promt)):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    outputs = torch.argmax(outputs, axis=1)   
    next_id = outputs[0][len(promt)+i-1]

    promt_ids[len(promt)+i] = next_id.item()
    print(promt_ids)

[3, 7, 5, 1, 1, 1, 1]
[3, 7, 5, 12, 1, 1, 1]
[3, 7, 5, 12, 10, 1, 1]
[3, 7, 5, 12, 10, 4, 1]
[3, 7, 5, 12, 10, 4, 13]
